In [2]:
import math
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt

def load_json_file(filename):
    """Carga un archivo JSON y lo retorna como diccionario."""
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            return json.load(file)
    except Exception as e:
        print(f"Error al cargar el archivo {filename}: {str(e)}")
        return None

def extract_unique_courses(profesores_data, scenario : str):
    """Extrae las asignaturas únicas manteniendo el orden original y sumando horas."""
    asignaturas = {}
    
    for profesor in profesores_data:
        for asignatura in profesor['Asignaturas']:
            codigo = asignatura['CodigoAsignatura']
            if codigo not in asignaturas:
                asignaturas[codigo] = {
                    'CodigoAsignatura': codigo,
                    'Nombre': asignatura['Nombre'],
                    'Vacantes': asignatura['Vacantes'],
                    'Horas': asignatura['Horas']
                }
            else:
                # Sumar las horas si la asignatura aparece más de una vez
                asignaturas[codigo]['Horas'] += asignatura['Horas']
    
    return pd.DataFrame(list(asignaturas.values()))

def calculate_te(asignatura, salas_data, total_bloques, scenario : str):
    """Calcula el TE para una asignatura específica."""
    salas_utiles = [sala for sala in salas_data if sala['Capacidad'] >= asignatura['Vacantes']]
    bloques_disponibles = len(salas_utiles) * total_bloques
    
    if total_bloques * len(salas_data) > 0:
        te = bloques_disponibles / (total_bloques * len(salas_data))
    else:
        te = 0
        
    return {
        'Asignatura': f"{asignatura['CodigoAsignatura']} - {asignatura['Nombre']}",
        'Vacantes': asignatura['Vacantes'],
        'Horas_Semanales': asignatura['Horas'],
        'Num_salas_utiles': len(salas_utiles),
        'Bloques_disponibles': bloques_disponibles,
        'TE': round(te, 2)
    }

def calculate_te_hours(asignatura, salas_data, total_bloques):
    salas_utiles = [sala for sala in salas_data if sala['Capacidad'] >= asignatura['Vacantes']]
    bloques_disponibles = len(salas_utiles) * total_bloques
    bloques_necesarios = asignatura['Horas']
    
    if total_bloques * len(salas_data) > 0:
        # Factor base de disponibilidad
        te_base = bloques_disponibles / (total_bloques * len(salas_data))
        
        # Factor de complejidad por horas
        factor_complejidad = math.exp(-0.2 * bloques_necesarios)
        
        # TE ajustado
        te = te_base * factor_complejidad
    else:
        te = 0
        
    return {
        'Asignatura': f"{asignatura['CodigoAsignatura']} - {asignatura['Nombre']}",
        'Vacantes': asignatura['Vacantes'],
        'Horas_Semanales': asignatura['Horas'],
        'Num_salas_utiles': len(salas_utiles),
        'Bloques_disponibles': bloques_disponibles,
        'Bloques_necesarios': bloques_necesarios,
        'TE': round(te, 2)
    }

def process_all_courses(asignaturas_df, salas_data, total_bloques, scenario : str):
    """Procesa todas las asignaturas y calcula sus TEs."""
    return pd.DataFrame([
        calculate_te(row, salas_data, total_bloques, scenario) 
        for _, row in asignaturas_df.iterrows()
    ])

def plot_te_boxplot(resultados_df):
    plt.figure(figsize=(10, 6))
    plt.boxplot(resultados_df['TE'])
    plt.title('Distribución de TE por Asignaturas')
    plt.ylabel('TE (Tasa de Disponibilidad)')
    plt.grid(True, linestyle='--', alpha=0.7)
    
    # Add summary statistics
    stats = {
        'Media': resultados_df['TE'].mean(),
        'Mediana': resultados_df['TE'].median(),
        'Q1': resultados_df['TE'].quantile(0.25),
        'Q3': resultados_df['TE'].quantile(0.75)
    }
    
    y_pos = plt.ylim()[0]
    for i, (label, value) in enumerate(stats.items()):
        plt.text(1.3, y_pos + (i * 0.05), f'{label}: {value:.3f}', 
                horizontalalignment='left')
    
    plt.savefig('te_boxplot.png')
    plt.close()

def plot_te_scatter(resultados_df):
    """
    Crea un diagrama de dispersión que relaciona TE con el número de Vacantes.
    
    Args:
        resultados_df: DataFrame con los resultados del análisis TE
    """
    plt.figure(figsize=(12, 8))

    # Crear el scatter plot
    scatter = plt.scatter(resultados_df['Vacantes'], 
                         resultados_df['TE'],
                         alpha=0.6,
                         c='#00629B',  # Color IEEE
                         s=100)  # Tamaño de los puntos

    # Añadir línea de tendencia
    z = np.polyfit(resultados_df['Vacantes'], resultados_df['TE'], 1)
    p = np.poly1d(z)
    plt.plot(resultados_df['Vacantes'], 
             p(resultados_df['Vacantes']), 
             linestyle='--', 
             color='#A9A9A9',
             alpha=0.8)

    # Configurar el aspecto
    plt.title('Relación entre TE y Número de Vacantes', 
             fontsize=14, 
             pad=20)
    plt.xlabel('Número de Vacantes', fontsize=12)
    plt.ylabel('TE (Tasa de Disponibilidad)', fontsize=12)

    # Eliminar bordes superiores y derechos
    ax = plt.gca()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # Añadir cuadrícula
    plt.grid(True, linestyle='--', alpha=0.3)

    # Calcular y mostrar correlación
    corr = resultados_df['Vacantes'].corr(resultados_df['TE'])
    plt.text(0.02, 0.98, 
             f'Correlación: {corr:.2f}',
             transform=ax.transAxes,
             fontsize=10,
             bbox=dict(facecolor='white', alpha=0.8, edgecolor='none'))

    plt.tight_layout()
    plt.savefig('te_scatter.png', dpi=300, bbox_inches='tight')
    plt.close()

def plot_te_horizontal_bars(resultados_df):
    """
    Crea un gráfico de barras horizontal que muestra el TE de cada asignatura.
    Las asignaturas están ordenadas por TE de forma descendente.
    Incluye el número de inscritos entre paréntesis junto al nombre de la asignatura.
    
    Args:
        resultados_df: DataFrame con los resultados del análisis TE
    """
    # Ordenar el DataFrame por TE de forma descendente
    df_sorted = resultados_df.sort_values('TE', ascending=True)
    
    # Crear etiquetas combinando nombre de asignatura y vacantes
    labels = [f"{asig} ({vac})" for asig, vac in 
             zip(df_sorted['Asignatura'], df_sorted['Vacantes'])]
    
    # Configurar el tamaño de la figura basado en el número de asignaturas
    plt.figure(figsize=(15, max(8, len(labels) * 0.3)))
    
    # Crear barras horizontales
    bars = plt.barh(range(len(labels)), 
                   df_sorted['TE'],
                   color='#00629B',  # Color IEEE
                   alpha=0.7)
    
    # Configurar ejes
    ax = plt.gca()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    # Configurar etiquetas del eje Y
    plt.yticks(range(len(labels)), labels, fontsize=8)
    
    # Configurar etiquetas del eje X
    plt.xlabel('TE (Tasa de Disponibilidad)', fontsize=12)
    
    # Añadir título
    plt.title('Tasa de Disponibilidad (TE) por Asignatura', 
             fontsize=14, 
             pad=20)
    
    # Añadir valores de TE al final de cada barra
    for i, bar in enumerate(bars):
        width = bar.get_width()
        plt.text(width, 
                bar.get_y() + bar.get_height()/2,
                f' {width:.2f}',
                va='center',
                fontsize=8)
    
    # Añadir grid vertical
    plt.grid(axis='x', linestyle='--', alpha=0.3)
    
    # Ajustar márgenes y layout
    plt.margins(y=0.01)
    plt.tight_layout()
    
    # Guardar figura
    plt.savefig('te_horizontal_bars.png', 
                dpi=300,
                bbox_inches='tight',
                facecolor='white')
    plt.close()


def plot_te_bubble(resultados_df):
    """
    Crea un diagrama de burbujas que relaciona TE, Vacantes y Horas semanales.
    
    Args:
        resultados_df: DataFrame con los resultados del análisis TE
    """
    plt.figure(figsize=(12, 8))

    # Normalizar el tamaño de las burbujas
    size_scale = resultados_df['Horas_Semanales'] * 20  # Factor de escala para mejor visualización

    # Crear el diagrama de burbujas
    scatter = plt.scatter(resultados_df['Vacantes'],
                         resultados_df['TE'],
                         s=size_scale,
                         c=resultados_df['Horas_Semanales'],
                         cmap='YlOrRd',
                         alpha=0.6)

    # Añadir barra de color
    cbar = plt.colorbar(scatter)
    cbar.set_label('Horas Semanales', fontsize=10)

    # Configurar el aspecto
    plt.title('Relación entre TE, Vacantes y Horas Semanales', 
             fontsize=14, 
             pad=20)
    plt.xlabel('Número de Vacantes', fontsize=12)
    plt.ylabel('TE (Tasa de Disponibilidad)', fontsize=12)

    # Eliminar bordes superiores y derechos
    ax = plt.gca()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # Añadir cuadrícula
    plt.grid(True, linestyle='--', alpha=0.3)

    # Añadir leyenda de tamaños
    legend_elements = [
        plt.scatter([], [], s=5*20, c='gray', alpha=0.4, label='5 horas'),
        plt.scatter([], [], s=10*20, c='gray', alpha=0.4, label='10 horas'),
        plt.scatter([], [], s=20*20, c='gray', alpha=0.4, label='20 horas')
    ]
    plt.legend(handles=legend_elements, 
              title='Horas Semanales',
              bbox_to_anchor=(1.15, 0),
              loc='lower right')

    plt.tight_layout()
    plt.savefig('te_bubble.png', dpi=300, bbox_inches='tight')
    plt.close()

def print_summary(resultados_df, num_salas, bloques_dia, num_dias):
    """Imprime el resumen de los resultados."""
    print("\nAnálisis de TE (Disponibilidad de Franjas Horarias para el Evento):")
    print(f"\nNúmero total de salas: {num_salas}")
    print(f"Número total de bloques por día: {bloques_dia}")
    print(f"Número total de días: {num_dias}")
    print("\nEstadísticas de TE:")
    print(f"Promedio: {resultados_df['TE'].mean():.4f}")
    print(f"Mediana: {resultados_df['TE'].median():.4f}")
    print(f"Desviación estándar: {resultados_df['TE'].std():.4f}")
    print(f"\nCorrelación entre Horas Semanales y TE: {resultados_df['Horas_Semanales'].corr(resultados_df['TE']):.4f}")
    print("\nResultados detallados:")
    print(resultados_df[['Asignatura', 'Vacantes', 'Horas_Semanales', 'Num_salas_utiles', 'TE']].to_string(index=False))

def main():
    # Cargar datos
    profesores_data = load_json_file('../../agent_input/inputOfProfesores.json')
    salas_data = load_json_file('../../agent_input/inputOfSala.json')
    
    if not profesores_data or not salas_data:
        return
    
    # Constantes
    BLOQUES_DIA = 9
    NUM_DIAS = 5
    TOTAL_BLOQUES = BLOQUES_DIA * NUM_DIAS
    
    # Procesar datos
    asignaturas_df = extract_unique_courses(profesores_data)
    resultados_df = process_all_courses(asignaturas_df, salas_data, TOTAL_BLOQUES)
    #plot_te_boxplot(resultados_df)
    plot_te_scatter(resultados_df)
    plot_te_horizontal_bars(resultados_df)
    #plot_te_bubble(resultados_df)
    
    # Generar salidas
    print_summary(resultados_df, len(salas_data), BLOQUES_DIA, NUM_DIAS)
    
    # Guardar resultados
    resultados_df.to_csv('resultados_TE.csv', index=False)
    
def main_with_platform():
    for scenario in ['full', 'medium', 'small']:
        profesores_data = load_json_file(f'../../dataset/scenarios/{scenario}/profesores.json')
        salas_data = load_json_file(f'../../dataset/scenarios/{scenario}/salas.json')
        
        if not profesores_data or not salas_data:
            print(f"Error al cargar datos para {scenario}")
            continue
        
        # Constantes
        BLOQUES_DIA = 9
        NUM_DIAS = 5
        TOTAL_BLOQUES = BLOQUES_DIA * NUM_DIAS
        
        # Procesar datos
        asignaturas_df = extract_unique_courses(profesores_data, scenario)
        resultados_df = process_all_courses(asignaturas_df, salas_data, TOTAL_BLOQUES, scenario)
        
        # Guardar resultados
        resultados_df.to_csv(f'../../dataset/scenarios/{scenario}/resultados_TE.csv', index=False)
        
    

if __name__ == "__main__":
    main_with_platform()